# Transformers Agents 快速上手

**注意！`Transformers Agents`是一个实验性的随时可能发生变化的 API。由于 API 或底层模型可能发生变化，agents 返回的结果也会有所不同。**

Transformers 版本`v4.29.0`基于`tools`和`agents`概念构建。你可以在[此 Cola 链接](https://colab.research.google.com/drive/1c7MHD-T1forUPGcC_jlwsIptOzpG3hSj)中进行测试。

简而言之，它在Transformers之上提供了一个自然语言API：

- 定义了一组经过筛选的`tools`，并设计了一个`agents`来解读自然语言并使用这些工具。
- 具有很强的可扩展性。
- 筛选了一些相关的`tools`，但我们将向你展示如何通过社区开发的`tool`轻松地扩展系统。

## Agent & Tool

**Agents**

这里的 Agents 本质是一个大型语言模型，我们可以通过提示让它访问特定的工具集。
大型语言模型在生成小代码示例方面表现出色，因此这个API利用这一特点，通过提示LLM生成一个使用 tools 集合的小代码示例。然后，根据您给 Agents 的任务和 tools 的描述来完成此提示。

**Tools**

Tools 是有名称和描述的单个函数。然后，我们使用这些 tools 的描述来提示代理。通过提示，让 agent 理解并学会如何使用 tool 来执行查询语言中请求的操作。
注意，**tools 不是 pipelines**，因为 agent 能够通过提示编写更具原子化的 tools，旨在专注于一个非常简单的任务。而 pipelines 经常被重构，且通常是将多个任务合并为一个。

## 实例化 agent

要使用 `agent.run`，你需要实例化一个 agent。

请注意，实例化的 agent 本质上是一个大型语言模型（LLM），因此你略有变化的提示可能会产生完全不同的结果。所以请尽可能清晰地解释你要执行的任务。

我们支持`OpenAI`模型以及来自 BigCode 和 OpenAssistant 的开源替代方案。OpenAI 模型性能更好（但需要你拥有 OpenAI API 密钥，因此无法免费使用），Hugging Face 为 BigCode 和 OpenAssistant 模型提供了免费访问端点。

一开始请安装 `agents` 附加模块，以安装所有默认依赖项。

In [ ]:
pip install transformers[agents]

### OpenAI
要使用OpenAI模型，可以在安装`openai`库后实例化一个`OpenAiAgent`：

In [ ]:
pip install openai

In [ ]:
from transformers import OpenAiAgent

agent = OpenAiAgent(model="text-davinci-003", api_key="<your_api_key>")

### BigCode or OpenAssistant
要使用`BigCode`或`OpenAssistant`，需要先登录以访问`Inference API`再实例化`agent`：

In [ ]:
from huggingface_hub import login

login("<YOUR_TOKEN>")

In [ ]:
from transformers import HfAgent

# Starcoder
agent = HfAgent("https://api-inference.huggingface.co/models/bigcode/starcoder")
# StarcoderBase
# agent = HfAgent("https://api-inference.huggingface.co/models/bigcode/starcoderbase")
# OpenAssistant
# agent = HfAgent(url_endpoint="https://api-inference.huggingface.co/models/OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5")

此示例使用了目前Hugging Face免费提供的推理API。如果你有自己的推理端点用于此模型（或其他模型），你可以用你的URL替换上面的URL。

`StarCoder`和`OpenAssistant`可以免费使用，并且在简单任务上表现出色。然而，当处理更复杂的提示时就不再有效。如果你遇到这样的问题，我们建议尝试使用`OpenAI`模型，尽管遗憾的是它不是开源的，但它在目前情况下表现更好。

我们可以从这个新API实现的示例开始。在处理多模态任务时它尤其强大，因此让我们快速试着生成图像并大声朗读文本。

In [ ]:
# 说明以下图片
agent.run("Caption the following image", image=image)

In [ ]:
# 将以下文本大声朗读
agent.run("Read the following text out loud", text=text)

In [ ]:
# 外部文档回答
agent.run(
    "In the following `document`, where will the TRRF Scientific Advisory Council Meeting take place?",
    document=document,
)

## 单次执行 (run)

单次执行方法是使用 agent 的 `Agent.run`：

In [ ]:
agent.run("Draw me a picture of rivers and lakes.")

它会自动选择适合你要执行的任务的tool（或tools），并以适当的方式运行它们。它可以在同一指令中执行一个或多个任务（当你的指令越复杂时，agent失败的可能性就越大）。

In [ ]:
agent.run("Draw me a picture of the sea then transform the picture to add an island")

每个 `run()` 操作都是**独立的**，因此你可以通过多次连续运行 run() 执行不同的任务。

如果你想在多次执行之间保持同一状态或向agent传递非文本对象，**可以通过指定agent要使用的变量来实现**。例如，你可以生成有关河流和湖泊的第一幅图像，并要求模型通过执行以下操作向该图片添加一个岛屿：

In [ ]:
picture = agent.run("Generate a picture of rivers and lakes.")
updated_picture = agent.run("Transform the image in `picture` to add an island to it.", picture=picture)

当模型无法准确地理解你的请求和库中的工具时，例如：

In [ ]:
agent.run("Draw me the picture of a capybara swimming in the sea")

在这种情况下，模型会通过两种方式理解你的请求：

- 使用`text-to-image` 生成在大海中游泳的大水獭
- 使用`text-to-image`生成大水獭，然后使用`image-transformation`工具使其在大海中游泳

如果你想强制使用第一种方式来生成图片，可以通过将提示作为参数传递给它来实现，提供更为清晰明确的提示让模型理解你的请求：

In [ ]:
agent.run("Draw me a picture of the `prompt`", prompt="a capybara swimming in the sea")

## 基于交流的执行 (chat)

这种方法可以在不同指令之间保持同一状态，也可以接受参数，以便你可以传递非文本类型或特定的提示。基于交流的执行（chat）方式是使用 `Agent.chat`：

In [ ]:
agent.chat("Generate a picture of rivers and lakes")

In [ ]:
agent.chat("Transform the picture so that there is a rock in there")

## Transformers 中集成的 tools

我们确定了一组可以赋予这些 agent 强大能力的 tools。以下是我们在 transformers 中集成的 tools：

- 文档问答：给定一个图像格式的文档（例如PDF），回答该文档上的问题（Donut）
- 文本问答：给定一段长文本和一个问题，回答文本中的问题（Flan-T5）
- 无条件图像字幕：为图像添加字幕！（BLIP）
- 图像问答：给定一张图像，回答该图像上的问题（VILT）
- 图像分割：给定一张图像和一个提示，输出该提示的分割掩模（CLIPSeg）
- 语音转文本：给定一个人说话的音频录音，将演讲内容转录为文本（Whisper）
- 文本转语音：将文本转换为语音（SpeechT5）
- Zero-Shot文本分类：给定一个文本和一个标签列表，确定文本最符合哪个标签（BART）
- 文本摘要：总结长文本为一两句话（BART）
- 翻译：将文本翻译为指定语言（NLLB）

这些tools也可以手动使用，例如：

In [ ]:
from transformers import load_tool

tool = load_tool("text-to-speech")
audio = tool("This is a text to speech tool")

## 自定义工具

通过将工具的代码上传到 Hugging Face 空间或模型 repository，你可以直接通过 agent 使用 tools。我们已经添加了一些与transformers无关的tools到[huggingface-tools组织](https://huggingface.co/huggingface-tools)中，这一实现能够帮助用户快速创建和共享自定义tool：

- 文本下载器：从Web URL下载文本
- 文本到图像：根据提示生成图像，利用stable diffusion
- 图像转换：根据初始图像和提示修改图像，利用instruct pix2pix stable diffusion
- 文本到视频：根据提示生成小视频，利用damo-vilab

从一开始就一直在使用的文生图tool 是一个远程tool ，位于[huggingface-tools/text-to-image](https://huggingface.co/spaces/huggingface-tools/text-to-image)！我们将继续在此组织和其他组织上发布此类tool，以进一步增强此实现。

agents 默认可以访问存储在 [huggingface-tools](https://huggingface.co/huggingface-tools) 上的 tools。我们将在后续的指南中解释如何编写和共享自定义tools，以及如何利用Hub上存在的任何自定义tools。

## 代码生成

到目前为止，我们已经展示了如何使用 agents 来为你执行操作。但是，agents 仅使用 Python 解释器执行的代码。如果你想要在不同的环境中使用生成的代码，可以提示 agents 返回可以执行的程序代码，以及tools的定义和准确的导入信息。例如：

In [ ]:
agent.run("Draw me a picture of rivers and lakes", return_code=True)